# FIT5196 Assessment 2
#### Student Name: Sarath Gopinathan
#### Student ID: 30434904

Date: 06/10/2020

Version: 1.4

Environment: Python 3.8.5 and Jupyter notebook

Libraries used: 
* pandas to read and perform actions on the given files
* numpy to perform calculations
* SentimentIntensityAnalyzer to perform sentiment analysis
* math to find distance between 2 lat longs
* numpy to solve linear equations
* itertools to get all the permutations and combinations of products

### Table of Contents

* [Importing packages](#import_packages)
* [Reading data](#reading_data)
* [Gathering data to complete tasks](#gather_data)
* [Adding the missing data](#missing_data)
* [Types of anomalies](#types_of_anomalies)
* [Process of cleaning the dirty data](#process)
* [Identifying and removing outliers](#outliers)


### Importing packages <a class="anchor" id="import_packages"></a>

Importing all packages required to complete the tasks

In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import math
import numpy as np
import itertools 

### Reading data <a class="anchor" id="reading_data"></a>

The package pandas is used to read the the input files. We can also view basic info of all the files(non-null counts, datatypes) using the package. This gives us insights about the data.

In [ ]:
dirty_data = pd.read_csv("30434904_dirty_data.csv")
outliers = pd.read_csv("30434904_outlier_data.csv")
missing = pd.read_csv("30434904_missing_data.csv")
warehouses = pd.read_csv("warehouses.csv")

In [ ]:
dirty_data.head(40)

In [ ]:
dirty_data.info()

In [ ]:
outliers.head(40)

In [ ]:
outliers.info()

In [ ]:
missing.head(40)

In [ ]:
missing.info()

In [ ]:
warehouses.head()

In [ ]:
warehouses.info()

### Gathering data to complete tasks <a class="anchor" id="gather_data"></a>

Gathering required data which will be used all along the process to complete all the tasks.

1. Identifying unique products
2. Calculating prices for each of the unique products.
3. Finding closest warehouse.

In [ ]:
# used to create the list of coefficients to help solve the linear equations
def create_lhs(cart, products):
    
    current_lhs = []
    
    for i in range(0, len(products)):
        
        current_lhs.append(0)
       
    for i in range(0, len(cart)):
        
        tag = products.index(cart[i][0])
        
        current_lhs[tag] = cart[i][1]
    
    return current_lhs


In [ ]:
# used to find unique products
def find_products(data):
    
    products = []

    for i in range (0, data['order_id'].count()):

            cart = eval(data.iloc[i][4])

            for j in range (0, len(cart)):

                if(cart[j][0] not in products):

                    products.append(cart[j][0])

    return products

In [ ]:
# used to find unique products and their respective price . Returns a tuple with product and price in it.
def get_products_with_price():

    products_with_price = []
    lhs = []
    rhs = []

    products = find_products(outliers)

    for i in range(0, len(products)):

        flag = 0

        for j in range (0, outliers['order_id'].count()):

            cart = eval(outliers.iloc[j][4])

            for k in range(0, len(cart)):

                if(cart[k][0] == products[i]):

                    current_lhs = create_lhs(cart, products)

                    if(current_lhs not in lhs):

                        lhs.append(current_lhs)
                        rhs.append(outliers.iloc[j][5])
                        flag = 1
                        break

            if(flag == 1):
                flag = 0
                break

    finalLHS = np.array(lhs)
    finalRHS = np.array(rhs)

    price_list = np.linalg.inv(finalLHS).dot(finalRHS)

    for i in range(0,len(products)):

        product_price = (products[i], round(price_list[i]))
        products_with_price.append(product_price)

    return products_with_price

In [ ]:
#used to calculate and find the closest warehouse. Returns the closest warehouse and the distance as a dictionary.
def calc_distance(lat1, long1):
    
    closest_warehouse = {}
    
    for i in range (0, warehouses['names'].count()):

        lat2 = warehouses.iloc[i][1]
        long2 = warehouses.iloc[i][2]
    
        # Converts lat & long to spherical coordinates in radians.
        degrees_to_radians = math.pi/180.0

        # phi = 90 - latitude
        phi1 = (90.0 - lat1)*degrees_to_radians
        phi2 = (90.0 - lat2)*degrees_to_radians

        # theta = longitude
        theta1 = long1*degrees_to_radians
        theta2 = long2*degrees_to_radians

        cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + math.cos(phi1)*math.cos(phi2))
        
        #computes distance value using formula
        distance = round(math.acos(cos)*6378,4) #radius of the earth in km
        
        if(i == 0):
            closest_warehouse = {"warehouse": warehouses.iloc[i][0], "distance": distance}
        else:
            if(closest_warehouse.get("distance") > distance):
                closest_warehouse = {"warehouse": warehouses.iloc[i][0], "distance": distance}
                
    return closest_warehouse

In [ ]:
#used to find the product price for the respective product
def find_prod_price(prod):
    
    prod_list = get_products_with_price()
    
    for i in range(0,len(prod_list)):
        
        if(prod_list[i][0] == prod):
            return prod_list[i][1]

In [ ]:
#used to calculate the order_price of the cart
def calculate_order_price(cart):
    
    total_order_price = 0
    
    for i in range(0, len(cart)):
        
        prod = cart[i][0]
        
        price = find_prod_price(prod)
        
        total_order_price += price * cart[i][1]
        
#     print(total_order_price)
        
    return total_order_price

### Adding the missing data <a class="anchor" id="missing_data"></a>

There are a lot of missing data in the missing data file. We check for cells with empty values and then fill them up with respective values after performing calculation.

### Process to add the missing data <a class="anchor" id="process_missing_data"></a>

Identify all the empty cells in each of the columns and fill them up with the respective values after calculating them. The following is the process to fill all the empty cells with their respective values.

1. Since there are missing values under order_price, we need to calculate all the products and their respective prices.
2. Calculate the distance between the customer location and all the three warehouse and check the distance_to_nearest_warehouse and match the respective warehouse and fill all the empty nearest_warehouse cells with the value.
3. Calculate order_price by performing calculations according to shopping_cart after finding the product prices and fill the empty cells with the value.
4. Calculate delivery_charges by using the formula (order_total - (order_price - (order_price*(coupon_discount/100)))) and fill all the empty cells.
5. Calculate the order_total by using the formula (order_price + ((order_price - order_price*(coupon_discount/100))) + delivery_charges) and fill all the empty cells.
6. Calculate the distance between the customer location and all the three warehouse and check for the smallest one and fill the empty distance_to_nearest_warehouse cell with the respective values.
7. Perform sentiment analysis on latest_customer_review and fill empty is_customer_happy value cells with the respective value.

In [ ]:
#gets the prices
individual_products = get_products_with_price()

#changes data whereever it mismatches
for i in range (0, missing['order_id'].count()):
    
    closest_warehouse = calc_distance(missing.iloc[i][7], missing.iloc[i][8])   
    
    if(pd.isnull(missing.iloc[i][3])): 
        missing.at[i,'nearest_warehouse'] = closest_warehouse.get("warehouse")
        if(pd.isnull(missing.iloc[i][13])): 
            missing.at[i,"distance_to_nearest_warehouse"] = closest_warehouse.get("distance")
    
    if(pd.isnull(missing.iloc[i][5])):
        
        order_price = calculate_order_price(eval(missing.iloc[i][4]))
        missing.at[i,"order_price"] = order_price
        
    if(pd.isnull(missing.iloc[i][6])):
        
        delivery_charges = missing.iloc[i][10] - (missing.iloc[i][5] - ((missing.iloc[i][5]*missing.iloc[i][9])/100))
        missing.at[i,"delivery_charges"] = delivery_charges
        
    if(pd.isnull(missing.iloc[i][10])):
        
        order_total = (missing.iloc[i][5] - (missing.iloc[i][5]*(missing.iloc[i][9]/100))) + missing.iloc[i][6]
        missing.at[i,"order_total"] = order_total
        
    if(pd.isnull(missing.iloc[i][15])):
        
        analyser = SentimentIntensityAnalyzer()
        score = analyser.polarity_scores(dirty_data.iloc[i][14])
        
        if(score.get('compound') >= 0.05):
        
            missing.at[i,"is_happy_customer"] = 1
            
        else:
            
            missing.at[i,"is_happy_customer"] = 0   

#writing to new file
missing.to_csv(r'30434904_missing_data_solution.csv', index = False)

### Types of anomalies <a class="anchor" id="types_of_anomalies"></a>

Data Anomalies can be classified at a high level into three categories:

* Syntactical Anomalies: describe characteristics concerning the format and values used for representation of the entities. Syntactical anomalies such as: lexical errors, domain format errors, syntactical error and irregularities.

* Semantic Anomalies: hinder the data collection from being a comprehensive and non-redundant representation of the mini-world. These types of anomalies include: Integrity constraint violations, contradictions, duplicates and invalid tuples.

* Coverage Anomalies: decrease the amount of entities and entity properties from the mini-world that are represented in the data collection. Coverage anomalies are categorised as: missing values and missing tuples.


### Process of cleaning the dirty data <a class="anchor" id="process"></a>

There are different anomolies that are present in the given data. We know that 3 columns have correct data - coupon_discount, delivery_chargesand quantity values in the shopping_cart. Thus we should check each and every other column to confirm if an anomaly exists. We also know that there can excist only 1 anomaly in a row. The following process will be followed to clean the given data :

1. Check if all the order ids are unique.
2. All date must follow the format YYYY-MM-DD
3. Check if the customer lat long are correct. If not, switch them and check if they are correct.
4. Check if the seasons fall under - Spring, Summer, Autumn, Winter and check if the date correctly matches.
5. Perform sentiment analysis on latest_customer_review and modify is_customer_happy value if there is a change.
6. Check if the nearest warehouse and the distance is correct by calculating the distance between customer location and the warehouse locations.
7. Find the price of each item in shopping cart using the outlier dataset as it has no anomalies in it.
8. Now check if the order price matches the shopping_cart and make changes as required.
9. After this, verify the order_total by substituting the values in the given formul and make changes if required.
10. Check if is_expedited is TRUE or FALSE using the given conditions.


In [ ]:
# check if all the order_ids are unique.
count = dirty_data['order_id'].count()
uniq = dirty_data['order_id'].nunique()

if(count == uniq):
    print("All order ids are unique")

### Phase 1 Cleaning (Date, Season, is_happy_customer)

Here we check the date format, season value and we perform sentiment analysis to confirm if the value in is_happy_customer is correct. If they are found to be wrong, the changes will be made and the value in changes_made will be changed to 1 from 0.

In [ ]:
dirty_data.describe()
dirty_data.head()

In [ ]:
dirty_data["is_happy_customer"] = dirty_data["is_happy_customer"].astype(int)
for i in range (0, dirty_data['order_id'].count()):
    
#     correcting date format
    if("-" in dirty_data.iloc[i][2][0:4]):
        dirty_data.at[i,'date'] = pd.to_datetime(dirty_data.iloc[i][2]).strftime('%Y-%m-%d')
    
    month = dirty_data.iloc[i][2][5:7]
        
#     correcting season if it doesnt match the month
    if(month == "09" or month == "10" or month == "11"):
        if(dirty_data.iloc[i][11] != "Spring"):
            dirty_data.at[i,'season'] = "Spring"         
    elif(month == "12" or month == "01" or month == "02"):
        if(dirty_data.iloc[i][11] != "Summer"):
            dirty_data.at[i,'season'] = "Summer"
    elif(month == "03" or month == "04" or month == "05"):
        if(dirty_data.iloc[i][11] != "Autumn"):
            dirty_data.at[i,'season'] = "Autumn"
    elif(month == "06" or month == "07" or month == "08"):
        if(dirty_data.iloc[i][11] != "Winter"):
            dirty_data.at[i,'season'] = "Winter"
    
#     performing sentiment analysis to correct the is_happy_customer column values
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(dirty_data.iloc[i][14])
    
    if(score.get('compound') >= 0.05):
        
        if(dirty_data.iloc[i][15] == 0):
            dirty_data.at[i,"is_happy_customer"] = 1  
            
    else:
            
        if(dirty_data.iloc[i][15] == 1):
            dirty_data.at[i,"is_happy_customer"] = 0
            
phase_one_clean = dirty_data

phase_one_clean.head(50)

### Phase 2 Cleaning (Nearest warehouse, customer location, distance to nearest warehouse)

Initially we check if the customer lat long are valid.If they are not valid, check by swtiching the lat and long. Next, from the warehouse details and the given customer lat long, we compute the nearest warehouse and the distance to the nearest warehouse. Make the changes as required.

In [ ]:
for i in range (0, phase_one_clean['order_id'].count()):
    
#     checking if the lat longs are switched
    if(phase_one_clean.iloc[i][7] > 90 or phase_one_clean.iloc[i][7] < -90):
        temp_lat = phase_one_clean.iloc[i][7]
        phase_one_clean.at[i,'customer_lat'] = phase_one_clean.iloc[i][8]
        phase_one_clean.at[i,'customer_long'] = temp_lat
    elif(phase_one_clean.iloc[i][8] > 180 or phase_one_clean.iloc[i][8] < -180):
        temp_lat = phase_one_clean.iloc[i][7]
        phase_one_clean.at[i,'customer_lat'] = phase_one_clean.iloc[i][8]
        phase_one_clean.at[i,'customer_long'] = temp_lat
    
#     Calculates closest warehouse. Contains warehouse and distance.
    closest_warehouse = calc_distance(phase_one_clean.iloc[i][7], phase_one_clean.iloc[i][8])
    
#     check if the distance is not equal to the closest warehouse distance and change if required.
    if(phase_one_clean.iloc[i][13] != closest_warehouse.get("distance")):
        phase_one_clean.at[i,'distance_to_nearest_warehouse'] = closest_warehouse.get("distance")
        
#     check if the warehouse is same as closest warehouse and change if required.        
    if(phase_one_clean.iloc[i][3] != closest_warehouse.get("warehouse")):
        phase_one_clean.at[i,'nearest_warehouse'] = closest_warehouse.get("warehouse")

        
phase_two_clean = phase_one_clean

phase_two_clean.head(60)    

### Phase 3 Cleaning (shopping cart, order price, order total)

Since outliers data doesnt have any anomalies, we find each items price from the outliers data by solving linear equations. We then check if the shopping cart, order price match. Change the values accordingly and then calculate order total.

In [ ]:
def perm_cart(cart, order_price):
    
    prods = find_products(outliers)
#     prods = []
    cart_size = len(cart)       
    perm_list = list(itertools.permutations(prods,cart_size))
    
    for i in range (0,len(perm_list)):
        
        current_cart = []
        
        for j in range (0, len(perm_list[i])):
        
            temp = (perm_list[i][j], cart[j][1])

            current_cart.append(temp)

        current_cart_price = calculate_order_price(current_cart)
        
        if(current_cart_price == order_price):
                return current_cart
    
    return []
    
    

In [ ]:
products_with_price = get_products_with_price()
products_with_price

for i in range (0, phase_two_clean['order_id'].count()):
    
    order_price = calculate_order_price(eval(phase_two_clean.iloc[i][4]))
    
    if(phase_two_clean.iloc[i][5] != order_price):
              
        cart = perm_cart(phase_two_clean.iloc[i][4], phase_two_clean.iloc[i][5])
        
        if(len(cart) == 0):
            
            phase_two_clean.at[i,'order_price'] = order_price
        
        else:
            
            phase_two_clean.at[i,'shopping_cart'] = cart
            
    current_order_total = (phase_two_clean.iloc[i][5] - (phase_two_clean.iloc[i][5]*(phase_two_clean.iloc[i][9]/100))) + phase_two_clean.iloc[i][6]
    phase_two_clean.at[i,"order_total"] = current_order_total

phase_three_clean = phase_two_clean

### Writing the clean file into a new csv 

We now write the entire dataframe as a csv.

In [ ]:
phase_three_clean.to_csv(r'30434904_dirty_data_solution.csv', index = False)

### Identifying and removing outliers <a class="anchor" id="outliers"></a>

Identifying and removing outliers

In [ ]:
outliers.boxplot(column='delivery_charges',figsize=(10,10))

In [ ]:
outliers.boxplot(figsize=(10,10))

In [ ]:
outliers['delivery_charges'].describe()

In [ ]:
#identifying q1 for 0.25 - lq
q1 = outliers['delivery_charges'].quantile(0.25)
#identifying q1 for 0.75 - hq
q3 = outliers['delivery_charges'].quantile(0.75)
iqr = q3-q1 #Interquartile range
fence_low  = q1-1.5*iqr
fence_high = q3+1.5*iqr
without_outliers = outliers.loc[(outliers['delivery_charges'] > fence_low) & (outliers['delivery_charges'] < fence_high)]

### Writing the clean file into a new csv 

We now write the entire dataframe as a csv.

In [ ]:
without_outliers.to_csv(r'30434904_outliers_data_solution.csv', index = False)